In [43]:
PROBE_VOLUME_ML = 6

In [44]:
import os
import pandas as pd
import numpy as np

In [45]:
def load_dfs_from_path(path="example"):

    combined_df = []

    for entry in os.scandir("example"):  
        if entry.is_file() and entry.name.endswith(".csv"):
            df = pd.read_csv(entry.path)
            combined_df.append(df)

    combined_df = pd.concat(combined_df, ignore_index=True)

    return combined_df

In [46]:
df = load_dfs_from_path("example")

# Scaling
df['dlugoscmm'] = df['dlugoscmm']/1.49
df['szerokoscmm'] = df['szerokoscmm']/1.49
df['powierzchniamm'] = df['powierzchniamm']/2.235

# filtering the things that have high probability that are not bacteria
df = df[df['dlugoscmm'] >= 0.2]
df = df[df['szerokoscmm'] <= 1.5]

df['Pw'] = 3.14*(df['szerokoscmm']/2)**2+df['szerokoscmm']*(df['dlugoscmm']-df['szerokoscmm'])
df['R'] = df['Pw'] / df['powierzchniamm']
df['Dk'] = df['dlugoscmm']
df['Sk'] = df['szerokoscmm']
df['D/S'] = (df['Dk'] / df['Sk']).astype('float64')

bacteria_types = []
Dks = []
Sks = []
for idx,row  in df.iterrows():
    if row['R'] > 1.2:
        Dks.append(row['szerokoscmm'])
        Sks.append(row['Pw']*0.8)
        bacteria_types.append("Krzywe")
    else:
        Dks.append(row['dlugoscmm'])
        Sks.append(row['szerokoscmm'])
        if round(row['D/S'],15) > 1.5:
            bacteria_types.append("Pałeczki")
        else:
            bacteria_types.append("Ziarniaki")
df['bacteria_type'] = bacteria_types
df['Dk'] = Dks
df['Sk'] = Sks

df['Ob'] =(3.14*(df['Sk']**3)/6)+(3.14*((df['Sk']**2)/4)*(df['Dk']-df['Sk']))
df = df[df['Ob']>0]
df['bialko'] = 104.5 * (df['Ob']**0.59)
df['wegiel'] = 0.86 * df['bialko']


In [47]:
grouped = df.groupby("bacteria_type")
result_count = grouped.size().reset_index(name="count")
result = grouped["Ob"].mean().reset_index()

result['bialko'] = 104.5 * (result['Ob']**0.59)
result['wegiel'] = 0.86 * result['bialko']
result['count_in_1_ml']=((result_count['count']*48097.39)/10)/(PROBE_VOLUME_ML)
result['biomasa'] =(((104.5*result['Ob']**0.59)*0.86)*result['count_in_1_ml'])/1000000

In [48]:
bins = [0, 0.1, 0.2, 0.5, 1.0, float("inf")]
labels = ["<=0.1", "0.1–0.2", "0.2–0.5", "0.5–1.0", ">1.0"]

df["Ob_bucket"] = pd.cut(df["Ob"], bins=bins, labels=labels, right=True)

grouped = df.groupby(["bacteria_type", "Ob_bucket"])
result_bio_stats = grouped.size().reset_index(name="count")

total_bacteria_count = np.sum(result_bio_stats['count'])
total_bacteria_count_1_ml = ((np.sum(result_bio_stats['count'])*48097.39)/10)/(PROBE_VOLUME_ML)

result_bio_stats['count_in_1_ml']=((result_bio_stats['count']*48097.39)/10)/(PROBE_VOLUME_ML)
result_bio_stats['Ob'] = grouped['Ob'].mean().values
result_bio_stats['bio_diversity'] =((result_bio_stats['count_in_1_ml']+1)/total_bacteria_count_1_ml)*np.log10((result_bio_stats['count_in_1_ml']+1)/total_bacteria_count_1_ml)

shannon_index = np.sum(result_bio_stats['bio_diversity'])*-1

C:\Users\dawid\AppData\Local\Temp\ipykernel_21152\2678725267.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(["bacteria_type", "Ob_bucket"])


In [49]:
from openpyxl import Workbook
from openpyxl.styles import Border, Side, Alignment, Font, PatternFill

wb = Workbook()
ws = wb.active
ws.title = "Summary"

thin_side = Side(border_style="thin", color="000000")
border = Border(top=thin_side, left=thin_side, right=thin_side, bottom=thin_side)
fill_white = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
center = Alignment(horizontal="center", vertical="center", wrap_text=True)
left = Alignment(horizontal="left", vertical="center")

for row in ws.iter_rows(min_row=1, max_row=100, min_col=1, max_col=50):
    for cell in row:
        cell.fill = fill_white

merge_ranges = [
    ("B2:B4",), ("C2:C4",), ("D2:D4",), ("E2:E4",),
    ("F2:H2",), ("I2:K2",), ("L2:N2",), ("O2:O4",),
    ("F3:F4",), ("G3:G4",), ("H3:H4",),
    ("I3:I4",), ("J3:J4",), ("K3:K4",),
    ("L3:L4",), ("M3:M4",), ("N3:N4",)
]
for rng in merge_ranges:
    ws.merge_cells(rng[0])

ws["B2"] = f"Zbadano\n{PROBE_VOLUME_ML}\nml wody"
ws["C2"] = "w\n10\npolach"
ws["D2"] = "Liczba\nw wodzie\nw 1 ml"
ws["E2"] = "%"

ws["F2"] = "Długość"
ws["F3"] = "średnia\nµm"
ws["G3"] = "min\nµm"
ws["H3"] = "max\nµm"

ws["I2"] = "Szerokość"
ws["I3"] = "średnia\nµm"
ws["J3"] = "min\nµm"
ws["K3"] = "max\nµm"

ws["L2"] = "Objętość"
ws["L3"] = "średnia\nµm^3"
ws["M3"] = "min\nµm^3"
ws["N3"] = "max\nµm^3"

ws["O2"] = "Biomasa\nC bakt.\nµg/l"

# Header cells
for row in ws["B2:O3"]:
    for cell in row:
        cell.border = border
        cell.fill = fill_white
        cell.alignment = center
        cell.font = Font(bold=True)

# Data cells
for r in range(4, 24):
    for c in range(2, 16):
        cell = ws.cell(row=r, column=c)
        cell.border = border
        cell.fill = fill_white
        cell.alignment = left if c == 2 else center

row_labels = {
    5: "Ogółem",
    6: "Pałeczki",
    7: "<0.1", 8: "0.1-0.2", 9: "0.2-0.5", 10: "0.5-1.0", 11: ">1.0",
    12: "Ziarniaki",
    13: "<0.1", 14: "0.1-0.2", 15: "0.2-0.5", 16: "0.5-1.0", 17: ">1.0",
    18: "Krzywe",
    19: "<0.1", 20: "0.1-0.2", 21: "0.2-0.5", 22: "0.5-1.0", 23: ">1.0",
}
bold_labels = {"Ogółem", "Bakterie", "Pałeczki", "Ziarniaki", "Krzywe"}

for r, text in row_labels.items():
    cell = ws.cell(row=r, column=2, value=text)
    cell.alignment = left
    if text in bold_labels:
        cell.font = Font(bold=True)


col_widths = {
    "B": 14, "C": 14, "D": 14, "E": 6,
    "F": 10, "G": 10, "H": 10,
    "I": 10, "J": 10, "K": 10,
    "L": 10, "M": 10, "N": 10,
    "O": 14
}
for col, w in col_widths.items():
    ws.column_dimensions[col].width = w

# Mini-table
mini_start_row = 26
ws.merge_cells(start_row=mini_start_row, start_column=10, end_row=mini_start_row, end_column=11)
for c in range(10, 12):
    cell = ws.cell(row=mini_start_row, column=c, value="Simon" if c == 10 else None)
    cell.font = Font(bold=True)
    cell.alignment = center
    cell.border = border
    cell.fill = fill_white

ws.cell(row=mini_start_row + 1, column=10, value="Węgiel\nfg/kom.").alignment = center
ws.cell(row=mini_start_row + 1, column=11, value="Białko\nfg/kom.").alignment = center
for c in (10, 11):
    cell = ws.cell(row=mini_start_row + 1, column=c)
    cell.font = Font(bold=True)
    cell.alignment = center
    cell.border = border
    cell.fill = fill_white

ws.merge_cells(start_row=mini_start_row, start_column=12, end_row=mini_start_row + 1, end_column=12)
cell = ws.cell(row=mini_start_row, column=12, value="Wskaźnik\nShannona")
cell.alignment = center
cell.font = Font(bold=True)
cell.border = border
cell.fill = fill_white

mini_labels = ["Bakterie", "Pałeczki", "Ziarniaki", "Krzywe"]
for i, label in enumerate(mini_labels, start=2):
    row_idx = mini_start_row + i
    cell = ws.cell(row=row_idx, column=9, value=label)
    cell.alignment = left
    cell.font = Font(bold=True)
    cell.border = border
    cell.fill = fill_white
    for c in (10, 11):
        cell2 = ws.cell(row=row_idx, column=c)
        cell2.border = border
        cell2.fill = fill_white
        cell2.alignment = center
    if i == 2:
        cell3 = ws.cell(row=row_idx, column=12)
        cell3.border = border
        cell3.fill = fill_white
        cell3.alignment = center

skip_cells = {(26, 9), (27, 9), (29, 12), (30, 12), (31, 12)}
for r in range(mini_start_row, mini_start_row + 6):
    for c in range(9, 13):
        cell = ws.cell(row=r, column=c)
        if (r, c) in skip_cells:
            cell.fill = fill_white
            continue
        cell.border = border


In [50]:
def populate_main_table(row, dataset):
    if dataset is not None and not dataset.empty:
        ws.cell(row=row, column=6, value=round(np.average(dataset["dlugoscmm"]), 4))
        ws.cell(row=row, column=7, value=round(np.min(dataset["dlugoscmm"]), 4))
        ws.cell(row=row, column=8, value=round(np.max(dataset["dlugoscmm"]), 4))
        ws.cell(row=row, column=9, value=round(np.average(dataset["szerokoscmm"]), 4))
        ws.cell(row=row, column=10, value=round(np.min(dataset["szerokoscmm"]), 4))
        ws.cell(row=row, column=11, value=round(np.max(dataset["szerokoscmm"]), 4))
        ws.cell(row=row, column=12, value=round(np.average(dataset["Ob"]), 4))
        ws.cell(row=row, column=13, value=round(np.min(dataset["Ob"]), 4))
        ws.cell(row=row, column=14, value=round(np.max(dataset["Ob"]), 4))
    else:
        ws.cell(row=row, column=6, value=0)
        ws.cell(row=row, column=7, value=0)
        ws.cell(row=row, column=8, value=0)
        ws.cell(row=row, column=9, value=0)
        ws.cell(row=row, column=10, value=0)
        ws.cell(row=row, column=11, value=0)
        ws.cell(row=row, column=12, value=0)
        ws.cell(row=row, column=13, value=0)
        ws.cell(row=row, column=14, value=0)

In [51]:
def split_ob_buckets(df, buckets):
    return [
        df[df['Ob'] <= buckets[0]],
        df[(df['Ob'] > buckets[0]) & (df['Ob'] <= buckets[1])],
        df[(df['Ob'] > buckets[1]) & (df['Ob'] <= buckets[2])],
        df[(df['Ob'] > buckets[2]) & (df['Ob'] <= buckets[3])],
        df[df['Ob'] > buckets[3]]
    ]

In [52]:
bacteria_types = ["Ziarniaki", "Pałeczki", "Krzywe"]
buckets = [0.1, 0.2, 0.5, 1.0]

# Create groups for each bacteria type
bacteria_groups = {}
for b in bacteria_types:
    bacteria_groups[b] = df[df['bacteria_type'] == b]

# Split each group into Ob buckets
bacteria_buckets = {}
for b in bacteria_types:
    bacteria_buckets[b] = split_ob_buckets(bacteria_groups[b], buckets)

row_map = {
    ("Bakterie", None): 5,
    ("Pałeczki", None): 6,
    ("Ziarniaki", None): 12,
    ("Krzywe", None): 18,
}

# Main bacteria summary rows
for idx, row in result.iterrows():
    bact = row["bacteria_type"]
    excel_row = row_map.get((bact, None))
    dataset = bacteria_groups.get(bact)
    if excel_row:
        for col in range(3, 16):
            ws.cell(row=excel_row, column=col).font = Font(bold=True)
        ws.cell(row=excel_row, column=15, value=round(row["biomasa"], 2))
        ws.cell(row=excel_row, column=4, value=round(np.sum(row["count_in_1_ml"]), 0))
        ws.cell(row=excel_row, column=5, value=round(row["count_in_1_ml"] / total_bacteria_count_1_ml * 100, 2))
        populate_main_table(excel_row, dataset)

# Total bacteria summary
for col in range(3, 16):
    ws.cell(row=5, column=col).font = Font(bold=True)
ws.cell(row=5, column=3, value=total_bacteria_count)
ws.cell(row=5, column=4, value=round(total_bacteria_count_1_ml, 0))
ws.cell(row=5, column=5, value=100.0)
populate_main_table(5, df)
ws.cell(row=5, column=15, value=round(((((104.5 * np.average(df["Ob"]) ** 0.59) * 0.86) * total_bacteria_count_1_ml) / 1_000_000), 2))

# Count rows
for idx, row in result_count.iterrows():
    bact = row["bacteria_type"]
    excel_row = row_map.get((bact, None))
    if excel_row:
        ws.cell(row=excel_row, column=3, value=row["count"])

# Ob bucket rows
bucket_labels = ["<=0.1", "0.1–0.2", "0.2–0.5", "0.5–1.0", ">1.0"]
bucket_row_offsets = {
    "Pałeczki": 7,
    "Ziarniaki": 13,
    "Krzywe": 19
}
for idx, row in result_bio_stats.iterrows():
    bact = row["bacteria_type"]
    ob_bin = row["Ob_bucket"]
    if bact in bucket_row_offsets and ob_bin in bucket_labels:
        bucket_idx = bucket_labels.index(ob_bin)
        excel_row = bucket_row_offsets[bact] + bucket_idx
        dataset = bacteria_buckets[bact][bucket_idx]
        ws.cell(row=excel_row, column=3, value=row["count"])
        ws.cell(row=excel_row, column=4, value=round(row["count_in_1_ml"], 0))
        ws.cell(row=excel_row, column=5, value=round(row["count"] / total_bacteria_count * 100, 2))
        populate_main_table(excel_row, dataset)
        ws.cell(row=excel_row, column=15, value=round(np.average((((104.5 * row['Ob'] ** 0.59) * 0.86) * row["count_in_1_ml"]) / 1_000_000), 2))

# Mini summary table
mini_start_row = 26
mini_labels = ["Bakterie", "Pałeczki", "Ziarniaki", "Krzywe"]
for i, label in enumerate(mini_labels, start=2):
    row_idx = mini_start_row + i
    if label in result["bacteria_type"].values:
        r = result[result["bacteria_type"] == label].iloc[0]
        ws.cell(row=row_idx, column=10, value=round(r["wegiel"], 5))
        ws.cell(row=row_idx, column=11, value=round(r["bialko"], 5))
    if label == "Bakterie":
        cell = ws.cell(row=row_idx, column=12, value=round(shannon_index, 2))
        cell.font = Font(bold=True)
        bialko = 104.5 * (np.average(df["Ob"] ** 0.59))
        wegiel = 0.86 * bialko
        cell = ws.cell(row=row_idx, column=10, value=round(wegiel, 5))
        cell.font = Font(bold=True)
        cell = ws.cell(row=row_idx, column=11, value=round(bialko, 5))
        cell.font = Font(bold=True)

wb.save("wyniki.xlsx")
